# Cvičení 4

Tématem tohoto cvičení jsou přímé řešiče soustav lineárních rovnic (LU, LDLT, LDMT, Choleského dekompozice) a jejich výpočetní náročnosti.

## Výpočetní náročnost přímých řešičů 

Připomeňme, kolik aritmetických operací přibližně vyžadují přímé řešiče:
- LU rozklad bez pivotizace: $\frac{2}{3}m^3$
- LDMT rozklad: $\frac{2}{3}m^3$
- LDLT rozklad: $\frac{1}{3}m^3$
- Choleského rozklad: $\frac{1}{2}m^3$
- dopředná a zpětná substituce: $\frac{1}{2}m^2$

V první části cvičení porovnáme výpočetní časy vybraných metod z NumPy a SciPy v závislosti na velikosti vstupní matice.

In [2]:
# importujme potrebne knihovny

import numpy as np
import scipy.linalg
import time
import matplotlib.pyplot as plt

Následující funkce se stará o vykreslování grafu času rozkladu v závislosti na velikosti matice. Na vstupu má 
- pole `mat_sizes` s velikostmi testovaných matic,
- pole `times` stejné velikosti jako `mat_sizes` s časy rozkladů matic o příslušné velikosti,
- string s názvem řešiče, který slouží k vytvoření popisků grafu.

Kód si prostudujte a pokuste se pochopit jeho jednotlivé části. Poté buňku zavolejte, abyste jej mohli využít v další části.

In [3]:
# Vytvoříme funkci pro vykreslení grafu času řešení.

def plot_result(mat_sizes, times, solver_name):
    fig, ax = plt.subplots()

    # pro ilustraci proložíme časy graf funkce x^3, kde za x dosadíme velikosti 
    # faktorizovaných matic - nejdříve vypočítáme hodnoty na této křivce, později je vykreslíme
    compl = mat_sizes ** 3
    fit = compl[-1] / times[-1]
    compl = compl.astype(float)
    compl /= fit    
    compl += times[0]

    # Vykreslíme časy výpočtu v závislosti na velikosti matice
    ax.plot(mat_sizes, times, linewidth=2, label=solver_name)
    # Vykreslíme ilustrační kubickou funkci
    ax.plot(mat_sizes, compl, '--', color=[0.5, 0.5, 0.5], linewidth=2,
            label='$C_{1}m^{3}$')
    
    # Nastavíme postupně nadpis grafu, popisky os x a y
    ax.set_title('Complexity of the {} factorization'.format(solver_name), fontsize=16)
    ax.set_xlabel('Matrix size', fontsize=14)
    ax.set_ylabel('Decomposition time [s]', fontsize=14)

    # Nastavíme rozsah os x a y, zapneme mřížku, vykreslíme legendu
    ax.set_xlim([mat_sizes[0], mat_sizes[-1]])
    ax.set_ylim([0, np.max(times) * 1.1])
    ax.grid(True)
    ax.legend(loc='upper left', fontsize=12)

Nyní již můžeme přikročit k samotnému testování výpočetní náročnosti. V následují buňce si 
- nejdříve vygenerujeme pole s velikostmi testovaných matic od nejmenší po největší,
- vytvoříme pole, do kterých budeme ukládat časy rozkladů jednotlivých matic,
- ve smyčce projdeme přes všechny požadované rozměry matic,
    - vygenerujeme si náhodnou symetrickou pozitivně definitní matici (abychom mohli testovat Choleského rozklad) o požadovaném rozměru,
    - zavoláme postupně LU, LDLT a Choleského rozklad nad vygenerovanou maticí (vygooglete nebo zjistěte v dokumentaci, které metody a z které knihovny je třeba volat),
    - pokaždé uložíme čas řešení na příslušné místo v příslušném poli,
- pomocí výše vytvořené metody vykreslíme grafy časů řešení v závislosti na velikosti matice (porovnejte s vykreslenou kubickou funkcí),
- nakonec do jednoho grafu vykreslíme všechny časy, ať je můžeme navzájem porovnat. 

In [ ]:
# ÚKOL: Doplňte chybějící kód

# Test vypocetnich narocnosti primych resicu

# Vygenerujeme vektor s rozmery matic
min_m = 200
max_m = 2000
step = 200

# ÚKOL: Použijte metodu arange z NumPy, abyste vygenerovali pole celých čísel 
# mezi min_m a max_m (včetně) s krokem step
mat_sizes = np.arange(min_m, max_m+step, step) 

# Predalokujeme si vektory, ve kterych budeme ukladat casy operaci
# Zjistete, k cemu slouzi metoda zeros_like. Proc zde musime explicitne specifikovat datovy typ?
t_lu = np.zeros_like(mat_sizes, dtype=np.float64)
t_ldl = np.zeros_like(mat_sizes, dtype=np.float64)
t_chol = np.zeros_like(mat_sizes, dtype=np.float64)

for i in range(len(mat_sizes)):
    # Vygenerujeme nahodnou symetrickou pozitivne definitni matici
    m = mat_sizes[i]
    A = np.random.rand(m, m)
    A = 0.5*(A + A.T)           # zajisti symetrii
    A = A + m * np.eye(m)       # zajisti diagonalni dominanci (pozitivni def.)

    # LU rozklad
    st = time.time()
    # ÚKOL: doplňte volání funkce, která počítá LU rozklad
    P, L1, U = scipy.linalg.lu(A)
    t_lu[i] = time.time() - st

    # LDLT rozklad
    st = time.time()
    # ÚKOL: doplňte volání funkce, která počítá LDLT rozklad
    L2, D, perm = scipy.linalg.ldl(A)
    t_ldl[i] = time.time() - st

    # Choleskeho rozklad
    st = time.time()
    # ÚKOL: doplňte volání funkce, která počítá Choleského rozklad
    R = scipy.linalg.cholesky(A)
    t_chol[i] = time.time() - st

# Vykreslime casy reseni v zavislosti na velikosti matice pro jednotlive resice
plot_result(mat_sizes, t_lu, 'LU')
plot_result(mat_sizes, t_ldl, 'LDLT')
plot_result(mat_sizes, t_chol, 'Cholesky')

# Vykreslime casy LU, LDL a Choleskeho rozkladu do jednoho grafu
fig, ax = plt.subplots()
ax.grid(True)
ax.plot(mat_sizes, t_lu, linewidth=2, label='LU')
ax.plot(mat_sizes, t_ldl, linewidth=2, label='LDLT')
ax.plot(mat_sizes, t_chol, linewidth=2, label='Cholesky')

ax.set_title('Decomposition times', fontsize=16)
ax.set_xlabel('Matrix size [m]', fontsize=14)
ax.set_ylabel('Decomposition time [s]', fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.legend(loc='upper left', fontsize=14)
plt.show()

## Choleského rozklad

Choleského rozklad slouží k rozkladu symetrické pozitivně definitní matice $\mathsf{A}$ na tvar
$$\mathsf{A}=\mathsf{R}^T\mathsf{R},$$
kde $R$ je horní trojúhelníková matice. Můžeme jej popsat např. následujícím pseudokódem:

```
m = size(A)
R = zeros(m)
for k = 1, ..., m do
    w = A(k+1 : m, k)
    A(k+1 : m, k+1 : m) = A(k+1 : m, k+1 : m) - w*w.T/A(k, k)
    R(k,k) = sqrt(A(k,k))
    R(k, k+1 : m) = w.T / sqrt(A(k,k))
end for
```


In [ ]:
# ÚKOL: Podobně jako v kódu testující výpočetní náročnost si vygenerujte nějakou 
# testovací matici A, která je symetrická pozitivně definitní (např. symetrickou
# diagonálně dominantní matici).

m = 5
A = np.random.rand(m, m)
A = 0.5*(A + A.T)           # zajisti symetrii
A = A + m * np.eye(m) 

np.set_printoptions(precision=3) # pro lepší přehlednost
print(A)

In [ ]:
# Tip: než se pustíte do následujícího úkolu porovnejte, co vrací 
# w = A[k+1 : m, k] 
#vs 
# w = A[k+1 : m, [k]]
# a jak v závislosti na tom dopadne operace w @ w.T (chcete dostat matici, ne skalár)


In [8]:
# ÚKOL: Implementujte na základě předchozího algoritmu funkci my_chol, která má na vstupu
# symetrickou pozitivně definitní matici A a vrací R.

def my_chol(A):
   m, n = np.shape(A)
    
   if (m != n):
      raise Exception("Matice musí být čtvercová")
    
   R = np.zeros_like(A)

   for k in range(0, m):
      w = A[k+1 : m, [k]]
      A[k+1 : m, k + 1 : m] = A[k+1 : m, k + 1 : m] -  w@w.T / A[k, k]
      R[k, k] = np.sqrt(A[k, k])
      R[k, k+1 : m] = w.T / np.sqrt(A[k, k])
   
   R = np.triu(R)
   return R


In [ ]:
# Ověříme, zda je naše řešení správné.
R = my_chol(A)
A2 = R.T@R
print(A2 - A)

In [ ]:
# Bonus: Výše vykreslené grafy nemusí být příliš hladké, časy mohou skákat v závislosti např. na aktuálním
# vytížení systému. Upravte kód tak, aby se grafy kreslily z průměrů časů výpočtu při několika bězích.

# Domácí úkol č. 2

Vašim úkolem bude porovnat výkon vámi implementovaných metod pro LU rozklad a Choleského rozklad podobně, jako jsme porovnávali zabudované SciPy metody v první části tohoto cvičení. Vyjděte tedy z výše implementovaného kódu, který ve smyčce volá rozklady matic, a upravte jej tak, aby volal vaše metody pro LU a Choleského rozklad místo NumPy metod (volání LDLT rozkladu můžete zrušit). Metodu `plot_results` upravovat nemusíte.

**Tip**: Vzhledem k tomu, že vaše metody nejspíš budou méně efektivní než původně volané SciPy metody, budete muset zmenšit maximální velikost testovaných matic.

**Odevzdání**: Celý tento soubor s řešením zašlete na e-mail vyučujícího do 30. 3. 16:00. Jako předmět e-mailu uveďte "NLA1 DU2".

In [ ]:
# Zde si z minulého cvičení zkopírujte vámi implementovanou metodu pro LU faktorizaci


In [ ]:
# Zde zkopírujte kód z třetí buňky a upravte jej tak, aby volal vaše metody pro LU a Choleského faktorizaci
